In [76]:
import requests
import pandas as pd
import json



In [77]:
import os

FOURSQUARE_KEY = os.getenv('FOURSQUARE_API_KEY')

# Foursquare

Send requests to Foursquare with a small radius (1000m) for all the bike stations in Vancouver. 

In [78]:
#Load Vancouver bike station DF from csv
station_df = pd.read_csv('../data/station_df.csv')
station_df

,id,name,latitude,longitude,slots
0,00fa94ad698dc4a9e4d708d6fd32f294,Chilco & Barclay,49.291909,-123.140713,18
1,012d3e06901cc222b1c2cf0a2ace3a29,St George & Broadway,49.262321,-123.093060,14
2,029a505bd4422a1afd127987757f71a6,Britannia Parking Lot,49.275882,-123.071865,14
3,0438114d2e6b96118de69bc9775bb21e,Morton & Denman,49.288030,-123.142135,26
4,0459b7e93703980b853cd65a9dc60596,Thornton & National,49.273777,-123.092723,14
...,...,...,...,...,...
253,fbb4c06b719596c21f91fd51ef6d4710,7th & Heather,49.265089,-123.119425,18
254,fe0c57d04d6e682d284d501446095fc9,Union & Dunlevy,49.277595,-123.095830,16
255,fedff7a263c182df94bda7307059cc52,Richards & Helmcken,49.277141,-123.122589,18
256,fef69fb400210d861107a61db954d037,Keefer & Abbott,49.279821,-123.108020,28


In [81]:
#Define FourSquare API function that counts places for each category argument passed

def contact_four_square(row, category, radius=1000, limit=50):
    #Base FourSquare endpoint
    foursquare_endpoint = "https://api.foursquare.com/v3/places/search"

    #Authorization headers
    headers = {
        "Accept": "application/json",
        "Authorization": FOURSQUARE_KEY
    }

    #Include search paramters
    #Latitude and Longitude string for params    
    params = {
        'limit': limit,
        'radius': radius,
        'll': f"{row['latitude']},{row['longitude']}",
        'categories': category,
        'fields': 'fsq_id,name,rating,popularity,price,venue_reality_bucket',
    }

    #Contact API
    foursquare_response = requests.get(foursquare_endpoint, headers=headers, params=params)

    #Check url for endpoint with params
    # print(foursquare_response.url)

    foursquare_list = foursquare_response.json()['results']
    place_count = len(foursquare_list)

    return place_count



In [83]:
#Interesting category numbers
categories = {
    'public_art': 10047,
    'night_club': 10032,
    'bicycle_store': 17119,
    'coffee_shop': 13032,
    'fast_food': 13145,
    'museum': 10027,
    'grocery_store': 17069,
}

#Iterate through all category names, using the id to make API calls to FourSquare and apply to station dataframe
for cat_name, id in categories.items():
    station_df[cat_name] = station_df.apply(contact_four_square, axis=1, args=(id,))



Chilco & Barclay


Write full data frame to CSV to preserve data.

In [ ]:
#View station data frame with point of interest counts
#NOTE: categories have 50 MAX items due to limitations in API call
station_df

,id,name,latitude,longitude,slots,public_art,night_club,bicycle_store,coffee_shop,fast_food,museum,grocery_store
0,00fa94ad698dc4a9e4d708d6fd32f294,Chilco & Barclay,49.291909,-123.140713,18,3,4,9,45,21,2,23
1,012d3e06901cc222b1c2cf0a2ace3a29,St George & Broadway,49.262321,-123.093060,14,1,5,16,50,15,0,28
2,029a505bd4422a1afd127987757f71a6,Britannia Parking Lot,49.275882,-123.071865,14,0,3,5,45,10,0,46
3,0438114d2e6b96118de69bc9775bb21e,Morton & Denman,49.288030,-123.142135,26,1,5,8,41,25,2,27
4,0459b7e93703980b853cd65a9dc60596,Thornton & National,49.273777,-123.092723,14,0,4,5,50,16,11,47
...,...,...,...,...,...,...,...,...,...,...,...,...
253,fbb4c06b719596c21f91fd51ef6d4710,7th & Heather,49.265089,-123.119425,18,3,3,18,44,18,3,13
254,fe0c57d04d6e682d284d501446095fc9,Union & Dunlevy,49.277595,-123.095830,16,2,11,7,50,28,19,44
255,fedff7a263c182df94bda7307059cc52,Richards & Helmcken,49.277141,-123.122589,18,3,44,11,50,50,5,38
256,fef69fb400210d861107a61db954d037,Keefer & Abbott,49.279821,-123.108020,28,6,34,14,50,50,23,41


In [ ]:
#Write data frame to csv to preserve contents
station_df.to_csv('../data/foursquare_df.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating